In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from __future__ import print_function, division
from IPython.display import clear_output, Image, display, HTML

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [3]:
def my_loop():

    def condition(v,t,current_x,current_u):
        return tf.less(t,num_epochs)

    def body(v,t,current_x,current_u):
        #h1=(t/tf_tau1 - 1)*tf_lamb*tf_c
        h1=tf.cond(t_true,lambda:(t/tf_tau1 - 1)*tf_lamb*tf_c,lambda: - tf_tau2*tf_lamb*tf_c - tf_lamb*tf_c )
        y=tf.matmul(tf_W,current_x) 
        h2=-tf_eta*(y-tf_theta)
        next_u=current_u + h1+h2
        next_x=tf.sigmoid(tf_beta*next_u)
        #u_series.append(next_u)
        #x_series.append(next_x)
        #print(next_x)
        y1=tf.matmul(tf_W,current_x)
        y2=tf.matmul(tf_W,next_x)
        y3=tf.norm(y1-y2)
        #t=tf.add(t,1)
        v=tf.cond(tf.norm(y3)<eps,lambda:t,lambda:tf.constant(-10,tf.float32))
        t=tf.cond(tf.norm(y3)<eps,lambda:tf.constant(num_epochs+1,tf.float32),lambda: tf.add(t,1))
        current_u=next_u
        current_x=next_x
        return(v,t,current_x,current_u)

    loop_vars=[v,t,current_x,current_u]

    return(tf.while_loop(condition,body,loop_vars))

In [4]:
n=5
num_epochs=1000
graph = tf.Graph()

with graph.as_default():
    v=tf.constant(0,tf.float32)
    t=tf.constant(1,tf.float32)
    eps=tf.constant(2**(-12),tf.float32)
    t_true=tf.placeholder(tf.bool)

    tf_eta=tf.placeholder(tf.float32,[],name = 'eta')
    tf_lamb=tf.placeholder(tf.float32,[n**2,1],name = 'lambda')
    tf_tau1=tf.placeholder(tf.float32,[],name = 'tau1')
    tf_tau2=tf.placeholder(tf.float32,[n**2,1],name = 'tau2')
    tf_beta=tf.placeholder(tf.float32,[],name= 'beta')

    tf_W=tf.placeholder(tf.float32,[n**2,n**2],name='W')
    tf_theta=tf.placeholder(tf.float32,[n**2,1], name = 'theta')
    tf_c=tf.placeholder(tf.float32,[n**2,1], name = 'c')

#init_x=tf.placeholder(tf.float32,[n**2,1],name='x')
#init_u=tf.placeholder(tf.float32,[n**2,1],name='u')

    init_x=tf.Variable(tf.constant(0, shape=[n**2,1],dtype=tf.float32),name='x')
    init_u=tf.Variable(tf.constant(0, shape=[n**2,1],dtype=tf.float32),name='u')
    current_u=init_u
    current_x=init_x

    loop=my_loop()
    
show_graph(graph.as_graph_def())

Instructions for updating:
Colocations handled automatically by placer.
